In [2]:
import pandas as pd
import re

In [ ]:
data = pd.read_excel(r'D:\WarsawParkInfo\data\table.xlsx', index_col='park_id')
data.head(7)

In [336]:
# Formating of area to digit format
area = data['area']
pattern = re.compile(r'\d+,*\d*')
digit = area.str.findall(pattern)
number = digit.apply(lambda lst: lst[0])

In [ ]:
# the table of Warsaw district.
html_tables = pd.read_html(r'https://pl.wikipedia.org/wiki/Podzia%C5%82_administracyjny_Warszawy')
districts = html_tables[0]['Dzielnica']
districts.index = pd.RangeIndex(len(districts), name='district_id')
districts = districts.str.replace('-', ' ')

In [ ]:
# districts in park table
pattern = re.compile('|'.join(districts))
p_districts = data['position']
p_districts = p_districts.str.findall(pattern).apply(lambda lst: lst[0] if len(lst) >0 else None)
p_districts

In [ ]:
# districts for the park table.
districts_index = pd.Index(districts)
district_id = districts_index.get_indexer(p_districts)

In [352]:
# streets address of parks. 
pattern = re.compile(r',',re.UNICODE)
buffer = data['position'].str.split(pattern, n=1, expand=True)
buffer = buffer.ffill(axis=1).iloc[:, 1]
position = buffer.apply(lambda s: s.strip())

In [368]:
# writing the data to df.
data['area'] = number
data['district_id'] = district_id
data['position'] = position

In [ ]:
# To write the data to excel
# with pd.ExcelWriter(r'D:\WarsawParkInfo\data\table.xlsx', mode='w') as writer:
#     data.to_excel(writer, sheet_name='Park', index=False)
#     districts.to_excel(writer, sheet_name='district', index=True)

In [4]:
# Creating of district of park table.
# key: Mokotow: 0, Ochota: 10, Śródmieście: 8.
data = pd.read_excel(r'D:\WarsawParkInfo\data\table.xlsx')
p_district_id = data[['district_id', 'park_id']]

# drop the record of park Pole-Mokotowskie
head, tail = p_district_id[:55], p_district_id[56:]
new_records = pd.DataFrame({'park_id': [55, 55, 55], 'district_id': [0, 10, 8]})
districts_of_park = pd.concat([head, tail, new_records])

In [ ]:
# To write the data to excel.
# with pd.ExcelWriter(r'D:\WarsawParkInfo\data\buffer.xlsx') as writer:
#     districts_of_park.to_excel(writer, sheet_name='Districts_of_park', index=False) 

In [9]:
# Coordinate of Warsaw districts. 
df = pd.read_csv(r'D:\WarsawParkInfo\data\district.csv', sep=';')
districts_coordinate = df[['WKT', 'name']]

In [46]:
def replace_pl_letters(record:pd.Series) -> pd.Series:
    transfer_map = str.maketrans({'Ą':'A','Ć':'C','Ę':'E','Ł':'L','Ń':'N','Ó':'O','Ś':'S','Ź':'Z','Ż':'Z','ą':'a','ć':'c','ę':'e','ł':'l','ń':'n','ó':'o','ś':'s','ź':'z','ż':'z'})
    return record.str.translate(transfer_map)
def format_url(park):
    park = re.sub(r'\W', '-', park)
    return re.sub(r'--+', '-', park)

In [403]:
# Coordinate of the Warsaw parks.
df = pd.read_csv(r'D:\WarsawParkInfo\data\park.csv', sep=';')
q_park: pd.DataFrame
q_park = df['website']
q_park.index = df['full_id']

e_park = pd.read_excel(r'D:\WarsawParkInfo\data\table.xlsx', sheet_name='Park')
e_park = e_park.iloc[:, [0, 1]]

pattern = re.compile('|'.join(e_park['park_name']), flags=re.IGNORECASE)

the_parks = q_park.str.findall(pattern)
the_parks = the_parks.explode()
# index maches
matches = the_parks[~the_parks.isnull()]
pattern = re.compile('|'.join(matches), flags=re.IGNORECASE)

matches.loc['w524003875'] = pd.NA
matches = matches.dropna()

In [420]:
is_fonded = df[df['full_id'].isin(matches.index)]
is_fonded.index = pd.Index(e_park['park_name'].str.lower()).get_indexer(matches)
# [123, 151, 36, 46, 233, 45, 51]
not_fonded = df.iloc[[123, 151, 36, 46, 233, 45, 51], :]
not_fonded.index =  pd.Index([4, 17, 26, 51, 63, 65, 67])

result = pd.concat([is_fonded, not_fonded])
result.index.name = 'park_id'
result = result.iloc[:, [0, -1]]

In [421]:
with pd.ExcelWriter(r'D:\WarsawParkInfo\data\buffer.xlsx') as writer:
    result.to_excel(writer, index=True)

In [7]:
df = pd.read_csv('D:\WarsawParkInfo\qgis\subway.csv', sep=';', encoding='utf-8')

df

<>:1: SyntaxWarning: invalid escape sequence '\W'
<>:1: SyntaxWarning: invalid escape sequence '\W'
C:\Users\Acer\AppData\Local\Temp\ipykernel_2756\3909446001.py:1: SyntaxWarning: invalid escape sequence '\W'
  df = pd.read_csv('D:\WarsawParkInfo\qgis\subway.csv', sep=';', encoding='utf-8')


,WKT,WKT.1,full_id,osm_id,osm_type,railway,railway:signal:wrong_road:form,railway:signal:wrong_road,railway:signal:main:substitute_signal,railway:signal:minor:substitute_signal,...,railway:position,level,ref,railway:switch:electric,railway:local_operated,name:pl,name:de,subway,public_transport,name
0,POINT (500503.407847472 5787265.8359984),POINT (500503.407847472 5787265.8359984),n35121251,35121251,node,stop,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,stop_position,Świętokrzyska
1,POINT (500682.595176826 5786747.62733374),POINT (500682.595176826 5786747.62733374),n35121252,35121252,node,stop,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Centrum,Zentrum,yes,stop_position,Centrum
2,POINT (501045.041366268 5785336.94145642),POINT (501045.041366268 5785336.94145642),n35121253,35121253,node,stop,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,stop_position,Politechnika
3,POINT (498935.339341288 5790989.87084654),POINT (498935.339341288 5790989.87084654),n35121254,35121254,node,stop,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,stop_position,Plac Wilsona
4,POINT (500523.074725496 5784230.48486219),POINT (500523.074725496 5784230.48486219),n35121256,35121256,node,stop,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,stop_position,Pole Mokotowskie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949,POINT (504686.142538098 5775346.54030685),POINT (504686.142538098 5775346.54030685),n12478519974,12478519974,node,signal,NaN,NaN,NaN,NaN,...,NaN,NaN,Ky35,NaN,NaN,NaN,NaN,NaN,NaN,NaN
950,POINT (504609.493330754 5775458.98419381),POINT (504609.493330754 5775458.98419381),n12478519980,12478519980,node,switch,NaN,NaN,NaN,NaN,...,NaN,NaN,10,yes,no,NaN,NaN,NaN,NaN,NaN
951,POINT (504583.447168015 5775511.06919122),POINT (504583.447168015 5775511.06919122),n12478519986,12478519986,node,switch,NaN,NaN,NaN,NaN,...,NaN,NaN,8,yes,no,NaN,NaN,NaN,NaN,NaN
952,POINT (504659.464248004 5775399.65869866),POINT (504659.464248004 5775399.65869866),n12478524149,12478524149,node,signal,NaN,NaN,NaN,NaN,...,NaN,NaN,Ky14,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
valid = df[~df['name'].isna()]
len(valid)

78

In [25]:
# valid = valid[['WKT', 'name']]

In [26]:
# valid.to_csv('subway.csv', index=False, sep=';', encoding='utf-8')

In [5]:
df = pd.read_csv(r"D:\WarsawParkInfo\qgis\park.csv", sep=';', encoding='utf-8')


In [10]:
df = df.iloc[:,0:-3]
df

,park_id,WKT,name,attractions,area,rating
0,0,MULTIPOLYGON (((501915.000443663 5782496.79247...,Arkadia,"stawy parkowe, muzeum im. Xawerego Dunikowskie...",11,"4,7"
1,1,MULTIPOLYGON (((502084.490957222 5786010.98333...,Park im. Marszałka Edwarda Rydza-Śmigłego,"fontanna i układ wodny od strony ul .Rozbrat, ...",53,"4,6"
2,2,MULTIPOLYGON (((501439.016146714 5787696.28040...,Park Kazimierzowski,"stary drzewostan, piękny widok rozciągający si...",4,"4,5"
3,3,MULTIPOLYGON (((503298.252986718 5788344.83684...,Park Skaryszewski,"stary drzewostan, układ wodny, grota, plac zab...","49,56","4,7"
4,4,MULTIPOLYGON (((506620.266751632 5787443.39696...,Park Znicza,"plac zabaw, miejsce pamięci, fontannaa","1,9","4,6"
...,...,...,...,...,...,...
65,65,MULTIPOLYGON (((507746.041715992 5788000.14570...,"Park Pułkownika Jana Szypowskiego ""Leśnika""",korty tenisowe w ośrodku sportowym TKKF\nsąsie...,"5,24","4,4"
66,66,MULTIPOLYGON (((504002.074754928 5776914.05463...,Lasek Brzozowy,"dwa place zabaw, stojaki rowerowe, boisko spor...","3,64","4,3"
67,67,MULTIPOLYGON (((502103.161851899 5785296.90236...,Park Jazdów,NaN,"5,7","4,6"
68,68,MULTIPOLYGON (((500965.838832368 5788297.43877...,Skwer Herberta Clarka Hoovera,NaN,"0,46","4,8"


In [52]:
df = pd.read_csv(r"D:\WarsawParkInfo\qgis\park.csv", sep=';', encoding='utf-8')
del df['Unnamed: 7']

In [61]:
df.to_csv('test.csv')